## IMPORT libraly

In [132]:
from scipy.signal import filtfilt, kaiserord, firwin, freqz, lfilter
from scipy import stats, signal
from pylab import figure, clf, plot, xlabel, ylabel, xlim, ylim, title, grid, axes, show
from numpy import cos, sin, pi, absolute, arange
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

## Load train/val data and target data

In [138]:
data = pd.read_csv('/Users/Asus TUF/Desktop/data/train_set.csv')
slice_D = data[['train_set']]
slice_df = pd.DataFrame(data=slice_D)
slice_df.columns = ['train_set']
torch_train = torch.tensor(slice_df['train_set'].values)
torch_train = torch_train.float()
torch_train = torch.reshape(torch_train, (40, 1, 64))
print(torch_train)
print(torch_train.shape)

data3 = pd.read_csv('/Users/Asus TUF/Desktop/data/valid_set.csv')
valid_D = data3[['valid_set']]
valid_df = pd.DataFrame(data=valid_D)
valid_df.columns = ['valid_set']
torch_valid = torch.tensor(valid_df['valid_set'].values)
torch_valid = torch_valid.float()
torch_valid = torch.reshape(torch_valid, (40, 1, 64))
print(torch_valid)
print(torch_valid.shape)

data2 = pd.read_csv('/Users/Asus TUF/Desktop/data/test_data.csv')
test_D = data2[['test_data']]
test_df = pd.DataFrame(data=test_D)
test_df.columns = ['test_data']
torch_test = torch.tensor(test_df['test_data'].values)
torch_test = torch_test.float()
torch_test = torch.reshape(torch_test, (18, 1, 64))
print(torch_test)
print(torch_test.shape)

tensor([[[0.0713, 0.0768, 0.0821,  ..., 0.0538, 0.0513, 0.0501]],

        [[0.1287, 0.1122, 0.0982,  ..., 0.1070, 0.1173, 0.1295]],

        [[0.0507, 0.0557, 0.0609,  ..., 0.0628, 0.0576, 0.0553]],

        ...,

        [[0.0529, 0.0410, 0.0326,  ..., 0.0806, 0.0822, 0.0834]],

        [[0.0709, 0.0737, 0.0764,  ..., 0.0641, 0.0633, 0.0623]],

        [[0.0538, 0.0508, 0.0478,  ..., 0.0845, 0.0838, 0.0849]]])
torch.Size([40, 1, 64])
tensor([[[0.0691, 0.0707, 0.0725,  ..., 0.0753, 0.0670, 0.0596]],

        [[0.0447, 0.0422, 0.0408,  ..., 0.0520, 0.0564, 0.0611]],

        [[0.0520, 0.0564, 0.0611,  ..., 0.0634, 0.0571, 0.0526]],

        ...,

        [[0.0896, 0.0918, 0.0934,  ..., 0.3156, 0.3315, 0.3404]],

        [[0.0448, 0.0487, 0.0543,  ..., 0.0654, 0.0624, 0.0600]],

        [[0.0687, 0.0654, 0.0624,  ..., 0.0846, 0.0979, 0.1112]]])
torch.Size([40, 1, 64])
tensor([[[0.0423, 0.0462, 0.0526,  ..., 0.0324, 0.0322, 0.0323]],

        [[0.0527, 0.0458, 0.0407,  ..., 0.0373, 0.036

## Load class

In [142]:
data4 = pd.read_csv('/Users/Asus TUF/Desktop/data/class_tr.csv')
classtr_D = data4[['class_tr']]
classtr_df = pd.DataFrame(data=classtr_D)
classtr_df.columns = ['class_tr']
class_trva = torch.tensor(classtr_df['class_tr'].values)
print(class_trva)
print(class_trva.shape)

data5 = pd.read_csv('/Users/Asus TUF/Desktop/data/class_te.csv')
classte_D = data5[['class_te']]
classte_df = pd.DataFrame(data=classte_D)
classte_df.columns = ['class_te']
class_te = torch.tensor(classte_df['class_te'].values)
print(class_te)
print(class_te.shape)

tensor([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0])
torch.Size([40])
tensor([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0])
torch.Size([18])


## test convo

In [124]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# x2 = torch.randn(50,1,64)
# conv = nn.Conv1d(1, 16, 3, padding=1)
# conv2 = nn.Conv1d(16, 32, 3, padding=1)
# conv3 = nn.Conv1d(32, 64, 3, padding=1)
# m = nn.MaxPool1d(2, stride=2)
# #y = torch.from_numpy(trainD)
# x_con1 = conv(torch_train)
# x_pool1 = m(x_con1)
# x_con2 = conv2(x_pool1)
# x_pool2 = m(x_con2)
# x_con3 = conv3(x_pool2)
# x_pool3 = m(x_con3)
# print(x_pool3.size())

torch.Size([20, 64, 8])


In [146]:
trainD = torch_train 
trainV = torch_valid
target = class_trva
print(len(trainD))

40


## Create Model architecture

In [144]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 64x1 1-D signal tensor)
        self.conv1 = nn.Conv1d(1, 16, 3, padding=1)
        # convolutional layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv1d(16, 32, 3, padding=1)
        # convolutional layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv1d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool1d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 8, 256)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(256, 2)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(-1, 64 * 8)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x

# create a complete CNN
model = Net()
print(model)


Net(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [145]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

## Train Model and validate model

In [151]:
# number of epochs to train the model
n_epochs = 30

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for i in range(40):
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(trainD)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*trainD.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for i in range(40):
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(trainV)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*trainV.size(0)
    
    # calculate average losses
    train_loss = train_loss/40
    valid_loss = valid_loss/40
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 5.124582 	Validation Loss: 4.038838
Validation loss decreased (inf --> 4.038838).  Saving model ...
Epoch: 2 	Training Loss: 3.932902 	Validation Loss: 3.063720
Validation loss decreased (4.038838 --> 3.063720).  Saving model ...
Epoch: 3 	Training Loss: 3.193080 	Validation Loss: 2.397420
Validation loss decreased (3.063720 --> 2.397420).  Saving model ...
Epoch: 4 	Training Loss: 2.651924 	Validation Loss: 1.955165
Validation loss decreased (2.397420 --> 1.955165).  Saving model ...
Epoch: 5 	Training Loss: 2.267624 	Validation Loss: 1.537340
Validation loss decreased (1.955165 --> 1.537340).  Saving model ...
Epoch: 6 	Training Loss: 1.875794 	Validation Loss: 1.284744
Validation loss decreased (1.537340 --> 1.284744).  Saving model ...
Epoch: 7 	Training Loss: 1.669447 	Validation Loss: 1.100001
Validation loss decreased (1.284744 --> 1.100001).  Saving model ...
Epoch: 8 	Training Loss: 1.439147 	Validation Loss: 0.972375
Validation loss decreased (1.10000

## Load the Model with the Lowest Validation Loss

In [152]:
model.load_state_dict(torch.load('model_cifar.pt')) #

<All keys matched successfully>

In [154]:
testD = torch_test
target = class_te
print(len(testD))
print(testD.shape)
print(target2.shape)
classes = ['1','0']

18
torch.Size([18, 1, 64])
torch.Size([18])


## Test Model

In [157]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))

model.eval()
# iterate over test data
for j in range(18):
    # move tensors to GPU if CUDA is available
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(testD)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*testD.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) 
    # calculate test accuracy for each object class
    for i in range(18):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/18
print('Test Loss: {:.6f}\n'.format(test_loss))


Test Loss: 0.226732



In [158]:
for i in range(2):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of     1: 100% (162/162)
Test Accuracy of     0: 100% (162/162)

Test Accuracy (Overall): 100% (324/324)
